In [120]:
# Bibliotecas

import pandas as pd
import os
from sqlalchemy import create_engine, inspect, text
from io import BytesIO
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta
from utils import sql_to_dbml # Importando a função sql_to_dbml do arquivo utils.py

In [121]:
# Dicionários e variáveis
db_path_silver = "../database/silver/01_silver.db"
dbml_path = "../modelagem/01_silver.dbml"

## Criação das tabelas

In [122]:
# Cria conexão com banco de dados SQLite
engine = create_engine(f"sqlite:///{db_path_silver}")

In [123]:
# Cria as tabelas no banco de dados

create_scripts = [
    """
    CREATE TABLE IF NOT EXISTS dim_tempo (
        id_data INTEGER PRIMARY KEY,
        data DATE,
        dia INTEGER,
        mes INTEGER,
        ano INTEGER,
        nome_mes TEXT,
        dia_da_semana TEXT,
        trimestre INTEGER
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS dim_cliente (
        id_cliente INTEGER PRIMARY KEY,
        nome_cliente TEXT,
        whatsapp TEXT
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS dim_vendedor (
        id_vendedor INTEGER PRIMARY KEY,
        nome_vendedor TEXT
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS dim_parcelamento (
        id_parcelamento INTEGER PRIMARY KEY,
        num_parcelas INTEGER,
        valor_parcela REAL,
        valor_total REAL,
        data_primeira_parcela INTEGER,
        data_ultima_parcela INTEGER
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS fato_vendas_alucar (
        id_venda_alucar INTEGER PRIMARY KEY,
        id_cliente INTEGER,
        id_data INTEGER,
        valor_venda REAL,
        FOREIGN KEY(id_cliente) REFERENCES dim_cliente(id_cliente),
        FOREIGN KEY(id_data) REFERENCES dim_tempo(id_data)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS fato_vendas_consigcar (
        id_venda_consigcar INTEGER PRIMARY KEY,
        id_cliente INTEGER,
        tipo_produto TEXT,
        id_parcelamento INTEGER,
        id_vendedor INTEGER,
        id_data INTEGER,
        FOREIGN KEY(id_cliente) REFERENCES dim_cliente(id_cliente),
        FOREIGN KEY(id_parcelamento) REFERENCES dim_parcelamento(id_parcelamento),
        FOREIGN KEY(id_vendedor) REFERENCES dim_vendedor(id_vendedor),
        FOREIGN KEY(id_data) REFERENCES dim_tempo(id_data)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS fato_despesas (
        id_despesa INTEGER PRIMARY KEY,
        origem TEXT,
        categoria TEXT,
        nome_despesa TEXT,
        valor REAL,
        id_data INTEGER,
        FOREIGN KEY(id_data) REFERENCES dim_tempo(id_data)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS fato_faturamento_pagseguro (
        id_faturamento INTEGER PRIMARY KEY,
        id_data INTEGER,
        valor_faturado REAL,
        FOREIGN KEY(id_data) REFERENCES dim_tempo(id_data)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS fato_vendas_mensais_consigcar (
        id_vendedor INTEGER,
        id_data INTEGER,
        valor_total REAL,
        quantidade_vendas INTEGER,
        bonus_salario REAL,
        PRIMARY KEY (id_vendedor, id_data),
        FOREIGN KEY(id_vendedor) REFERENCES dim_vendedor(id_vendedor),
        FOREIGN KEY(id_data) REFERENCES dim_tempo(id_data)
    );
    """
]

# Executa cada comando separadamente
with engine.begin() as conn:
    for stmt in create_scripts:
        conn.execute(text(stmt))

In [124]:
# Transformar o banco de dados em DBML para acompanhamento da modelagem
dbml = sql_to_dbml(engine)

with open(dbml_path, "w", encoding="utf-8") as f:
    f.write(dbml)

## Carregamento e transformação dos dados da camada Bronze

In [125]:
# Paths dos bancos Bronze e Silver
db_path_bronze1 = "../database/bronze/00_base1.db"
db_path_bronze2 = "../database/bronze/00_base2.db"
db_path_silver  = "../database/silver/01_silver.db"

In [126]:
# Engines de conexão
engine_bronze1 = create_engine(f"sqlite:///{db_path_bronze1}")
engine_bronze2 = create_engine(f"sqlite:///{db_path_bronze2}")
engine_silver  = create_engine(f"sqlite:///{db_path_silver}")

In [127]:
# 1) dim_cliente
# Vem de: 00_base2.db."00_vendas_clientes_consigcar".Nome, .WhatsApp
df_clientes = pd.read_sql(
    'SELECT DISTINCT Nome AS nome_cliente, WhatsApp AS whatsapp '
    'FROM "00_vendas_clientes_consigcar"',
    engine_bronze2
)
df_clientes.to_sql("dim_cliente", engine_silver, if_exists="replace", index_label="id_cliente")

207

In [128]:
# 2) dim_vendedor
# Vem de: 00_base2.db.Vendedor
df_vend = pd.read_sql(
    'SELECT DISTINCT Vendedor AS nome_vendedor '
    'FROM "00_vendas_clientes_consigcar"',
    engine_bronze2
)
df_vend.to_sql("dim_vendedor", engine_silver, if_exists="replace", index_label="id_vendedor")

7

In [129]:
# 3) dim_tempo
# Gera calendário de 2018-01-01 a 2030-12-31
start = datetime(2018,1,1)
end   = datetime(2030,12,31)
dates = []
while start <= end:
    dates.append(start)
    start += relativedelta(days=1)

df_tempo = pd.DataFrame({
    "id_data": [int(d.strftime("%Y%m%d")) for d in dates],
    "data": dates,
})
df_tempo["dia"]           = df_tempo["data"].dt.day
df_tempo["mes"]           = df_tempo["data"].dt.month
df_tempo["ano"]           = df_tempo["data"].dt.year
df_tempo["nome_mes"]      = df_tempo["data"].dt.strftime("%B")
df_tempo["dia_da_semana"] = df_tempo["data"].dt.strftime("%A")
df_tempo["trimestre"]     = df_tempo["data"].dt.quarter

df_tempo.to_sql("dim_tempo", engine_silver, if_exists="replace", index=False)

4748

In [130]:
# 4) dim_parcelamento
# Vem de: 00_base2.db."00_vendas_clientes_consigcar"

df_parc = pd.read_sql(
    'SELECT rowid AS id_parcelamento, '
    '"Quantidade de vezes" AS num_parcelas, '
    '"Valor parcela" AS valor_parcela, '
    '"Data do Pagamento" AS data_pagamento '
    'FROM "00_vendas_clientes_consigcar"',
    engine_bronze2
)

# Limpeza prévia de strings suspeitas e conversão robusta de datas
df_parc["data_pagamento"] = df_parc["data_pagamento"].astype(str).str.extract(
    r'(\d{1,2}[/\-]\d{1,2}[/\-]\d{2,4})'
)[0]

# Converte a data do pagamento para formato AAAAMMDD
df_parc["data_primeira_parcela"] = (
    pd.to_datetime(df_parc["data_pagamento"], format="mixed", dayfirst=True, errors="coerce")
    .dt.strftime("%Y%m%d")
    .astype("Int64")
)

# Calcula a data da última parcela com base na quantidade de parcelas
df_parc["data_ultima_parcela"] = (
    pd.to_datetime(df_parc["data_pagamento"], format="mixed", dayfirst=True, errors="coerce")
    + pd.to_timedelta((df_parc["num_parcelas"] - 1) * 30, unit="D")  # Aproximação: 1 mês = 30 dias
)
df_parc["data_ultima_parcela"] = (
    df_parc["data_ultima_parcela"].dt.strftime("%Y%m%d").astype("Int64")
)

# Calcula o valor total da compra (parcela * quantidade)
df_parc["valor_total"] = (df_parc["num_parcelas"] * df_parc["valor_parcela"]).round(2)

# Exporta a tabela para a camada silver
df_parc[[
    "id_parcelamento", "num_parcelas", "valor_parcela",
    "valor_total", "data_primeira_parcela", "data_ultima_parcela"
]].to_sql("dim_parcelamento", engine_silver, if_exists="replace", index=False)


207

In [131]:
# 5) fato_despesas
# Vem de: 00_base1.db."00_despesas_alucar", "00_despesas_consigcar"

# Leitura dos dados de despesas da Alucar
df_alucar = pd.read_sql(
    'SELECT DESPESAS AS nome_despesa, Valor AS valor, Mês AS mes '
    'FROM "00_despesas_alucar"',
    engine_bronze1
)
df_alucar["origem"] = "Alucar"

# Leitura dos dados de despesas da ConsigCar
df_consig = pd.read_sql(
    'SELECT DESPESAS AS nome_despesa, Valor AS valor, Mês AS mes '
    'FROM "00_despesas_consigcar"',
    engine_bronze1
)
df_consig["origem"] = "ConsigCar"

# Junta ambas as tabelas
df_desp = pd.concat([df_alucar, df_consig], ignore_index=True)

# Extrai data robustamente e cria id_data no formato YYYYMM01
df_desp["mes"] = df_desp["mes"].astype(str).str.extract(
    r'(\d{1,2}[/\-]\d{2,4}|\d{4}[/\-]\d{1,2})'
)[0]
df_desp["id_data"] = (
    pd.to_datetime(df_desp["mes"], format="mixed", dayfirst=True, errors="coerce")
    .dt.strftime("%Y%m01")
    .astype("Int64")
)

# Coluna categoria (placeholder)
df_desp["categoria"] = "NA"  # Atualizar futuramente se a info vier do Excel

# Exporta para camada silver
df_desp[["origem", "categoria", "nome_despesa", "valor", "id_data"]].to_sql(
    "fato_despesas", engine_silver, if_exists="replace", index_label="id_despesa"
)


420

In [132]:
# 6) fato_faturamento_pagseguro
# Vem de: 00_base1.db."00_receita_pagseguro_consigcar"
df_pag = pd.read_sql(
    'SELECT Data AS data, Valor AS valor_faturado '
    'FROM "00_receita_pagseguro_consigcar"',
    engine_bronze1
)
df_pag["id_data"] = (
    pd.to_datetime(df_pag["data"], dayfirst=True)
    .dt.strftime("%Y%m%d")
    .astype(int)
)

df_pag[["id_data","valor_faturado"]].to_sql(
    "fato_faturamento_pagseguro", engine_silver, if_exists="replace", index_label="id_faturamento"
)

4

In [133]:
# 7) fato_vendas_alucar
# Vem de: 00_base1.db."00_vendas_clientes_alucar"
df_val = pd.read_sql(
    'SELECT rowid AS id_venda_alucar, '
    'Data AS data, '
    '"Valor Receita" AS valor_venda '
    'FROM "00_vendas_clientes_alucar"',
    engine_bronze1
)

# Tratamento da data para criar id_data
df_val["id_data"] = (
    pd.to_datetime(df_val["data"], errors="coerce", dayfirst=True)
    .dt.strftime("%Y%m%d")
    .astype("Int64")  # Usamos Int64 p/ permitir nulos se houver erro no parsing
)

df_val[["id_venda_alucar", "id_data", "valor_venda"]].to_sql(
    "fato_vendas_alucar", engine_silver, if_exists="replace", index=False
)


81

In [134]:
# 8) fato_vendas_consigcar
# Vem de: 00_base2.db."00_vendas_clientes_consigcar"
df_vc = pd.read_sql(
    'SELECT rowid AS id_venda_consigcar, '
    'Nome AS id_cliente, '
    '"Tipo Produto" AS tipo_produto, '
    'rowid AS id_parcelamento, '
    'Vendedor AS id_vendedor, '
    '"Data do Pagamento" AS data '
    'FROM "00_vendas_clientes_consigcar"',
    engine_bronze2
)

df_vc["id_data"] = (
    pd.to_datetime(df_vc["data"], errors="coerce", dayfirst=True)
    .dt.strftime("%Y%m%d")
    .astype("Int64")
)

df_vc[[
    "id_venda_consigcar", "id_cliente", "tipo_produto",
    "id_parcelamento", "id_vendedor", "id_data"
]].to_sql(
    "fato_vendas_consigcar",
    engine_silver,
    if_exists="replace",
    index=False
)


207

In [135]:
# 9) fato_vendas_mensais_consigcar
df_det = pd.read_sql("SELECT * FROM fato_vendas_consigcar", engine_silver)
df_grp = df_det.groupby(["id_vendedor","id_data"]).agg(
    valor_total=("id_venda_consigcar","count"),
    quantidade_vendas=("id_venda_consigcar","count")
).reset_index()

def classifica_meta(q):
    if q >= 42: return "Meta 2"
    if q >= 30: return "Meta 1"
    if q >= 18: return "Bônus 1"
    return "NA"

df_grp["bonus_meta"] = df_grp["quantidade_vendas"].apply(classifica_meta)
df_grp.to_sql("fato_vendas_mensais_consigcar", engine_silver, if_exists="replace", index=False)

49

In [136]:
# Transformar o banco de dados em DBML para acompanhamento da modelagem

dbml = sql_to_dbml(engine_silver)
with open("../modelagem/01_silver.dbml", "w", encoding="utf-8") as f:
    f.write(dbml)